In [1]:
import { useState, useEffect } from 'react';
import { Card, CardContent } from '@/components/ui/card';
import { Button } from '@/components/ui/button';
import { Input } from '@/components/ui/input';
import { Select, SelectItem } from '@/components/ui/select';
import { Label } from '@/components/ui/label';
import { Dialog, DialogTrigger, DialogContent } from '@/components/ui/dialog';
import { initializeApp } from 'firebase/app';
import { getAuth, signInWithPopup, GoogleAuthProvider, onAuthStateChanged, signOut } from 'firebase/auth';

const firebaseConfig = {
  apiKey: 'YOUR_FIREBASE_API_KEY',
  authDomain: 'YOUR_FIREBASE_AUTH_DOMAIN',
  projectId: 'YOUR_FIREBASE_PROJECT_ID',
  appId: 'YOUR_FIREBASE_APP_ID'
};

const app = initializeApp(firebaseConfig);
const auth = getAuth(app);
const provider = new GoogleAuthProvider();

export default function Home() {
  const [user, setUser] = useState(null);
  const [products, setProducts] = useState([]);
  const [filters, setFilters] = useState({ size: '', color: '' });
  const [form, setForm] = useState({ name: '', size: '', color: '', price: '', image: '' });
  const [deliveryInfo, setDeliveryInfo] = useState({ name: '', phone: '', address: '', method: '' });
  const [selectedProduct, setSelectedProduct] = useState(null);

  useEffect(() => {
    onAuthStateChanged(auth, setUser);
  }, []);

  const handleUpload = (e) => {
    const file = e.target.files[0];
    const imageUrl = URL.createObjectURL(file);
    setForm({ ...form, image: imageUrl });
  };

  const addProduct = () => {
    if (!user) return alert('Please log in to upload');
    const newProduct = { ...form, id: Date.now() };
    setProducts([...products, newProduct]);
    setForm({ name: '', size: '', color: '', price: '', image: '' });
  };

  const handleLogin = () => signInWithPopup(auth, provider);
  const handleLogout = () => signOut(auth);

  const handlePayment = (price) => {
    const handler = window.PaystackPop && window.PaystackPop.setup({
      key: 'YOUR_PAYSTACK_PUBLIC_KEY',
      email: user?.email || 'test@example.com',
      amount: parseInt(price.replace('₦', '')) * 100,
      currency: 'NGN',
      callback: function (response) {
        alert('Payment successful! Ref: ' + response.reference + '\nDelivery info saved.');
        console.log('Delivery Details:', deliveryInfo);
      },
      onClose: function () {
        alert('Transaction was not completed.');
      }
    });
    handler?.openIframe();
  };

  const filteredProducts = products.filter(
    (product) =>
      (!filters.size || product.size === filters.size) &&
      (!filters.color || product.color === filters.color)
  );

  return (
    <div className="p-6 space-y-6">
      <h1 className="text-3xl font-bold text-center">Aso Ebi Marketplace 🇳🇬</h1>

      {user ? (
        <div className="text-right">
          <p>Welcome, {user.displayName}</p>
          <Button onClick={handleLogout}>Logout</Button>
        </div>
      ) : (
        <div className="text-right">
          <Button onClick={handleLogin}>Login with Google</Button>
        </div>
      )}

      {/* Upload Product Section */}
      <div className="space-y-4">
        <h2 className="text-2xl font-semibold">Upload Your Aso Ebi</h2>
        <Input placeholder="Name" value={form.name} onChange={(e) => setForm({ ...form, name: e.target.value })} />
        <Input placeholder="Price (₦)" value={form.price} onChange={(e) => setForm({ ...form, price: e.target.value })} />
        <Select onValueChange={(value) => setForm({ ...form, size: value })}>
          <SelectItem value="">Select Size</SelectItem>
          <SelectItem value="S">Small</SelectItem>
          <SelectItem value="M">Medium</SelectItem>
          <SelectItem value="L">Large</SelectItem>
        </Select>
        <Select onValueChange={(value) => setForm({ ...form, color: value })}>
          <SelectItem value="">Select Color</SelectItem>
          <SelectItem value="Green">Green</SelectItem>
          <SelectItem value="Red">Red</SelectItem>
          <SelectItem value="Blue">Blue</SelectItem>
        </Select>
        <Input type="file" accept="image/*" onChange={handleUpload} />
        {form.image && <img src={form.image} alt="Uploaded" className="w-32 h-32 object-cover rounded" />}
        <Button onClick={addProduct}>Submit</Button>
      </div>

      {/* Filter Section */}
      <div className="grid grid-cols-1 md:grid-cols-2 gap-4">
        <div>
          <Label htmlFor="size">Filter by Size</Label>
          <Select onValueChange={(value) => setFilters({ ...filters, size: value })}>
            <SelectItem value="">All</SelectItem>
            <SelectItem value="S">Small</SelectItem>
            <SelectItem value="M">Medium</SelectItem>
            <SelectItem value="L">Large</SelectItem>
          </Select>
        </div>
        <div>
          <Label htmlFor="color">Filter by Color</Label>
          <Select onValueChange={(value) => setFilters({ ...filters, color: value })}>
            <SelectItem value="">All</SelectItem>
            <SelectItem value="Green">Green</SelectItem>
            <SelectItem value="Red">Red</SelectItem>
            <SelectItem value="Blue">Blue</SelectItem>
          </Select>
        </div>
      </div>

      {/* Product List */}
      <div className="grid grid-cols-1 md:grid-cols-3 gap-6">
        {filteredProducts.map((product) => (
          <Card key={product.id}>
            <CardContent className="space-y-2">
              {product.image && <img src={product.image} alt={product.name} className="w-full h-48 object-cover rounded" />}
              <h2 className="text-xl font-semibold">{product.name}</h2>
              <p>Size: {product.size}</p>
              <p>Color: {product.color}</p>
              <p>Price: {product.price}</p>
              <Dialog>
                <DialogTrigger asChild>
                  <Button onClick={() => setSelectedProduct(product)}>Buy Now</Button>
                </DialogTrigger>
                <DialogContent className="space-y-4">
                  <h3 className="text-lg font-bold">Delivery Information</h3>
                  <Input placeholder="Full Name" value={deliveryInfo.name} onChange={(e) => setDeliveryInfo({ ...deliveryInfo, name: e.target.value })} />
                  <Input placeholder="Phone Number" value={deliveryInfo.phone} onChange={(e) => setDeliveryInfo({ ...deliveryInfo, phone: e.target.value })} />
                  <Input placeholder="Address" value={deliveryInfo.address} onChange={(e) => setDeliveryInfo({ ...deliveryInfo, address: e.target.value })} />
                  <Select onValueChange={(value) => setDeliveryInfo({ ...deliveryInfo, method: value })}>
                    <SelectItem value="">Select Delivery Method</SelectItem>
                    <SelectItem value="Bike">Bike Delivery</SelectItem>
                    <SelectItem value="Car">Car Delivery</SelectItem>
                    <SelectItem value="Pickup">Self Pickup</SelectItem>
                  </Select>
                  <Button onClick={() => handlePayment(product.price)}>Confirm & Pay</Button>
                </DialogContent>
              </Dialog>
            </CardContent>
          </Card>
        ))}
      </div>
    </div>
  );
}


SyntaxError: invalid character '🇳' (U+1F1F3) (<ipython-input-1-88c93a7c3105>, line 75)